Here, we are working on MNIST dataset
spikejelly library is used

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import spikingjelly.activation_based as sj

# Convert images to tensors and binarize them
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x > 0.5)])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [8]:
import spikingjelly.activation_based.neuron as neuron

class SNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)  # First layer (input to hidden)
        self.lif1 = neuron.LIFNode(tau=2.0)  # Corrected LIF neuron import
        self.fc2 = nn.Linear(128, 10)  # Output layer
        self.lif2 = neuron.LIFNode(tau=2.0)  # Corrected LIF neuron import
    
    def forward(self, x):
        x = x.view(x.shape[0], -1)  # Flatten images
        x = self.fc1(x)
        x = self.lif1(x)
        x = self.fc2(x)
        x = self.lif2(x)
        return x


In [12]:
# Initialize model, optimizer, and loss function
model = SNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(torch.float)  # Ensure images are float
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Reset neuron states to prevent accumulation
        for module in model.modules():
            if isinstance(module, neuron.LIFNode):  # Ensure LIF neurons reset
                module.reset()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")


Epoch 1, Loss: 1.5960, Accuracy: 79.39%
Epoch 2, Loss: 1.5156, Accuracy: 90.09%
Epoch 3, Loss: 1.5033, Accuracy: 92.00%
Epoch 4, Loss: 1.4957, Accuracy: 93.32%
Epoch 5, Loss: 1.4904, Accuracy: 94.14%
